In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from langchain_openai import ChatOpenAI
from utils import load_dataset
from langchain_core.prompts import PromptTemplate
import re
import json
import random
from tqdm.notebook import tqdm




In [3]:
random.seed(42)
scenes, questions = load_dataset()
random.shuffle(questions)

In [4]:
def process_scene(scene):
    if "image_index" in scene:
        scene.pop("image_index")

    for obj in scene["objects"]:
        if "rotation" in obj:
            obj.pop("rotation")
        if "pixel_coords" in obj:
            obj.pop("pixel_coords")
        if "3d_coords" in obj:
            obj.pop("3d_coords")
    
    if "image_filename" in scene:
        scene.pop("image_filename")

    if "split" in scene:
        scene.pop("split")

    if "directions" in scene:
        scene.pop("directions")

def get_question_by_category(questions, category, count=20):
    filtered_questions = []

    for question in questions:
        if category == "count" and question["answer"].isdigit():
            filtered_questions.append(question)
        elif category == "judge" and question["answer"].lower() in ["yes", "no"]:
            filtered_questions.append(question)
        elif category == "query" and question["answer"].lower() not in ["yes", "no"] and not question["answer"].isdigit():
            filtered_questions.append(question)
        
        if len(filtered_questions) >= count:
            break
    
    return filtered_questions

In [198]:
category = "query"
filtered_questions = get_question_by_category(questions, category, count=50)
filtered_dataset = []

In [311]:
# count_skip_index = [2, 15, 24, 34, 36, 37, 38, 39]
# # wrong answer
# # wrong answer
# # not answerable without image
# judge_skip_index = [1, 3, 4, 5, 34]
# not answerable without image
# wrong answer
# not answerable without image
# the question is a bit ambiguous
# typo in the question
query_skip_index = [4, 5, 11, 15, 18, 19, 29, 33, 38, 39, 42, 43, 44, 45, 46]
# not answerable without image
# not answerable without image
# wrong answer
# wrong answer
# not answerable without image
# not answerable without image
# not answerable without image
# not answerable without image
# not answerable without image
# not answerable without image
# not answerable without image
# wrong answer
# not answerable without image
# not answerable without image
# not answerable without image

In [315]:
id = 48
print(filtered_questions[id])


{'split': 'val', 'image_filename': 'CLEVR_val_000654.png', 'answer': 'cyan', 'question': 'What color are the two identical objects?', 'image_index': 654}


In [316]:
process_scene(scenes[filtered_questions[id]["image_index"]])
scenes[filtered_questions[id]["image_index"]]

{'objects': [{'color': 'red',
   'size': 'small',
   'shape': 'sphere',
   'material': 'rubber'},
  {'color': 'cyan',
   'size': 'large',
   'shape': 'cylinder',
   'material': 'rubber'},
  {'color': 'yellow',
   'size': 'large',
   'shape': 'cylinder',
   'material': 'rubber'},
  {'color': 'green',
   'size': 'small',
   'shape': 'cylinder',
   'material': 'rubber'},
  {'color': 'red', 'size': 'large', 'shape': 'cube', 'material': 'metal'},
  {'color': 'blue', 'size': 'large', 'shape': 'cylinder', 'material': 'metal'},
  {'color': 'cyan',
   'size': 'large',
   'shape': 'cylinder',
   'material': 'rubber'}],
 'relationships': {'right': [[1, 2, 3, 4, 6],
   [2, 3, 4, 6],
   [6],
   [2, 6],
   [2, 3, 6],
   [0, 1, 2, 3, 4, 6],
   []],
  'behind': [[1, 6],
   [],
   [0, 1, 3, 5, 6],
   [0, 1, 6],
   [0, 1, 2, 3, 5, 6],
   [0, 1, 3, 6],
   [1]],
  'front': [[2, 3, 4, 5],
   [0, 2, 3, 4, 5, 6],
   [4],
   [2, 4, 5],
   [],
   [2, 4],
   [0, 2, 3, 4, 5]],
  'left': [[5],
   [0, 5],
   [0, 1

In [317]:
processed_questions = []

for i, question in enumerate(filtered_questions[0:id+1]):
    if i in query_skip_index:
        continue
    processed_questions.append(question)

In [318]:
len(processed_questions)

34

In [319]:
with open("data/filtered_questions.jsonl", "a") as f:
    for question in processed_questions:
        f.write(json.dumps(question) + "\n")